In [16]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
from IPython.display import clear_output

from tqdm.auto import tqdm

import numpy as np
import pandas as pd
import os

In [17]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

import random

In [18]:
driver = webdriver.Chrome()

In [19]:
# naver 주소 접속
driver.get('https://www.naver.com')

In [20]:
# 검색창에 키워드 입력
keyword = '김현지'
driver.find_element(By.ID, 'query').send_keys('김현지')

In [21]:
# 검색 버튼 클릭
driver.find_element(By.CSS_SELECTOR,'#sform > fieldset > button').click()

In [22]:
# 뉴스 탭 클릭
driver.find_element(By.LINK_TEXT,'뉴스').click()

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=143.0.7499.41); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#staleelementreferenceexception
Stacktrace:
0   chromedriver                        0x0000000105322e10 cxxbridge1$str$ptr + 3028012
1   chromedriver                        0x000000010531acd0 cxxbridge1$str$ptr + 2994924
2   chromedriver                        0x0000000104e16b1c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74196
3   chromedriver                        0x0000000104e2746c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 142116
4   chromedriver                        0x0000000104e26568 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 138272
5   chromedriver                        0x0000000104e1d158 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 1003

In [ ]:
def check_exists_by_selector(driver, selector):
    try:
        driver.find_element(By.CSS_SELECTOR, selector)
    except NoSuchElementException:
        return False
    return True

In [23]:
data_dict = {
            'title' : [],
            'date' : [],
            'reporter' : [],
            'content' : []
}

# 네이버 뉴스 클릭 후 데이터 추출하기
box = driver.find_element(By.CLASS_NAME, 'group_news')

lists = box.find_elements(By.LINK_TEXT,'네이버뉴스')

for li in tqdm(lists):
    news_url = li
    # news_url 을 찾아서 그 위치로 이동
    actions = ActionChains(driver).move_to_element(news_url)
    actions.perform()

    # 클릭시 새로운 탭으로 전환 (window: Keys.CONTROL, Mac: Keys.COMMAND)
    news_url.send_keys(Keys.COMMAND + '\n')
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(3)

    title = driver.find_element(By.CSS_SELECTOR, '#ct > div.media_end_head.go_trans > div.media_end_head_title').text.strip()
    data_dict['title'].append(title)
    date = driver.find_element(By.CSS_SELECTOR, '#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_journalist > div > div.media_end_head_info_datestamp > div > div > div:nth-child(1) > span').text.strip()
    data_dict['date'].append(date)
    if check_exists_by_selector(driver, '#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_journalist > div > div.media_end_head_journalist_name_area'):
        reporter = driver.find_element(By.CSS_SELECTOR, '#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_journalist > div > div.media_end_head_journalist_name_area').text.strip()
    else:
        reporter = ''
    data_dict['reporter'].append(reporter)
    content = driver.find_element(By.CSS_SELECTOR, '#contents').text.strip()
    data_dict['content'].append(content)

    driver.close()
    driver.switch_to.window(driver.window_handles[0]) 

  0%|          | 0/23 [00:00<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
# DataFrame 만들기
df = pd.DataFrame(data_dict)

In [ ]:
# csv 저장
df.to_csv(f'navernews_{keyword}.csv')